In [1]:
from IPython.display import Image

In [3]:
# https://medium.com/@gxyang13/close-back-door-for-causal-models-a-guide-to-causal-graph-d4483cd5a276
Image(url='https://miro.medium.com/v2/resize:fit:1100/format:webp/1*MHt9cPbTLxofdbo11HpgqA.png', width=400)

- 链 (Chain): 想象一个多米诺骨牌效应。X 的发生导致 Y 的发生，Y 的发生又导致 Z 的发生。信息或影响沿着箭头的方向单向流动。例如，不良饮食习惯 (X) 可能导致高血糖 (Y)，而高血糖 (Y) 可能导致糖尿病 (Z)。
- 分叉 (Fork): 想象一个共同的原因。Y 是 X 和 Z 共同的“父亲”或原因。Y 的发生会同时影响 X 和 Z。例如，一个人的基因 (Y) 可能同时影响其身高 (X) 和患某种遗传病的风险 (Z)。变量 Y 在这里是一个混杂因子 (confounding variable)，因为它同时影响 X 和 Z  。
- V型结构 (V-structure / Collider): 想象两个独立的原因共同导致一个结果。X 和 Z 都是 Y 的“父母”或原因，它们共同影响 Y，但 X 和 Z 之间没有直接的因果关系。例如，一个学生的才华 (X) 和努力程度 (Z) 共同决定了其考试成绩 (Y)。在这种结构中，Y 被称为对撞节点 (collider)  。一个重要的特性是，在未控制 Y 或其任何后代节点时，X 和 Z 是独立的；但如果控制了 Y（比如只看考试成绩优秀的学生），X 和 Z 之间可能会变得相关（例如，在成绩优秀的学生中，才华较高的学生可能努力程度较低，反之亦然）。

### bidirectional-edge/arcs

- 如果研究者怀疑任意两个变量的误差项是相关的（例如，这两个变量有一个未被观察到或潜在的共同原因），那么就在它们之间画一条双向弧线
    - 如果我们观察到“鞋子尺码 (X)”和“阅读能力 (Y)”在儿童中相关，这并不是因为鞋子大导致阅读能力强，也不是反过来。而是因为存在一个共同的原因——“年龄 (U)”。年龄增长既导致脚变大，也导致阅读能力提高 (X←U→Y)。如果我们不把年龄 U 放入模型中，就可以用一条双向边 X↔Y 来表示这种由未观测的年龄因素造成的关联。

### d-separation

- d分离的规则：d-分离是判断在给定节点集 $Z$ 的条件下，两组变量是否独立的重要准则
    - 如果在一个因果图中，所有链接 $x$ 和 $z$ 的路径都被阻断（blocked），那么 $x$ 和 $z$ 在给定一组变量 $S$ 的条件下是独立的；
- 一条路径被阻断的条件是
    - (1) 链或分叉被阻塞(linear/fork) ：路径包含一个链结构 (A -> B -> C) 或 分叉结构 (A <- B -> C)，并且中间节点 B 在条件集 S 中。
        - 想象在一条水管（路径）的中间加了一个阀门（控制了 $m$ ），水流（信息）就过不去了。 
    - (2) 对撞结构与阻塞（collider）：路径包含一个对撞结构/汇聚点 (A -> B <- C)，并且中间节点 B 以及 B 的任何后代 都不在条件集 S 中。
        - 对撞: i -> m <- j (例如，才华 -> 奖学金 <- 努力)
        - 这种情况非常特殊，其阻断条件与链/分叉相反。
        - 如果路径 p 上存在这样的对撞结构，并且中间节点 m 不在集合 Z 中，并且 m 的任何后代节点 (descendant) 也不在集合 Z 中，那么这条路径就被 Z 在 m 处阻断了。
        - 直观理解 (非常重要): 当两个独立的原因 i 和 j 共同导致一个结果 m 时（对撞），如果我们不观察 m 或 m 的任何后果，那么 i 和 j 之间通常是没有关联的（路径是阻断的）。但是，如果我们观察了 m (即 m 在 Z 中)，或者观察了 m 的某个后代 (即 m 的某个后代在 Z 中)，那么这条路径就会被打开，i 和 j 之间就会产生关联（称为“选择偏差”或“对撞偏差”）。例如，如果只看获得奖学金的学生（控制了 m），那么才华和努力之间可能会呈现负相关，因为在获得奖学金的人群中，才华不高的人必然非常努力，反之亦然。
- d-分离是判断在给定一组变量 Z 的条件下，另外两个变量 X 和 Y 是否条件独立的方法。如果 X 和 Y 之间所有的路径都被 Z 阻断，那么 X 和 Y 就是 d-分离的（条件独立的）。

### blocked path 

- blocked 相对的概念是 open

$$
\begin{split}
p(x,z|y)&=p(x|y)p(z|y)\\
&\updownarrow \\
p(z|x,y)&=p(z|y)
\end{split}
$$
- 简单证明
$$
p(z|x,y)=\frac{p(x,y,z)}{p(x,y)}=\frac{p(x,z|y)}{p(x|y)}=\frac{p(x|y)p(z|y)}{p(x|y)}=p(z|y)
$$
- linear chain（链式结构）：$x\rightarrow y \rightarrow z$
    - $x\perp \!\!\! \perp z \mid y$（$x$ 和 $z$ 在给定 $y$ 的条件下独立），conditional independent
    - X 导致 Y，Y 导致 Z。信息从 X 流向 Y，再从 Y 流向 Z。
    - 不控制 Y 时: X 和 Z 是相关的（依赖的）。因为 X 的变化会通过 Y 影响 Z。
    - 控制 Y 时 (给定 Y): 当我们观察或固定了 Y 的值时，Y 就“阻断”了 X 到 Z 的路径。知道了 Y 之后，X 对于预测 Z 不再提供通过 Y 传递的额外信息。
    $$
    \begin{split}
    p(x,y,z)&=p(z|y)p(y|x)p(x)\\
    p(x,y,z)&=p(z|y)p(x|y)p(y)\\
    \frac{p(x,y,z)}{p(y)}&=p(z|y)p(x|y)\\
    p(x,z|y)&=p(z|y)p(x|y)\\
    \end{split}
    $$
- fork/comman cause（分叉结构）：$x\leftarrow y \rightarrow z$
    - Y 是 X 和 Z 的共同原因。Y 导致 X，Y 也导致 Z。
    - 不控制 Y 时: X 和 Z 是相关的（依赖的）。这是因为它们共享一个共同的原因 Y。例如，如果 Y 增加了 X 和 Z 同时增加的可能性，那么 X 和 Z 就会表现出相关性，即使它们之间没有直接的因果关系（这种相关性有时被称为“伪相关”）。
- collider/common effect（对撞结构）：$x\rightarrow y\leftarrow z$
    - X 和 Z 都是 Y 的原因（Y 是 X 和 Z 的共同结果）。路径在 Y 点“对撞”。
    - 一个具体的例子，$X\sim U[0,10], Z\sim U[0, 10], Y=X+Z$
        - 此时，given $Y=15$，显然 $X, Z$ 是不独立的

### Backdoor path 、frontdoor path

- 给定一个有序变量对 $(X,Y)$，后门路径是指任何从 $X$ 到 $Y$ 的路径中，起始于一个指向 $X$ 的箭头。这条后门路径是从 $X$ 到 $Y$ 的一条非因果路径。
    - 如果我们要研究“吃冰淇淋 ($X$ ) 是否会导致溺水 ($Y$)”，可能会观察到两者正相关。但实际上，炎热天气 ($W$) 既会导致人们吃更多冰淇淋 (W→X)，也会导致更多人去游泳从而增加溺水风险 (W→Y)。那么，X←W→Y 就是一条从 X 到 Y 的后门路径（路径为 X←W→Y，其中指向 X 的箭头是 W→X）。这条路径使得冰淇淋和溺水看起来相关，但并非因果关系。 为了得到 X 对 Y 的真实因果效应，我们需要通过控制合适的变量集 Z 来“关闭”或“阻塞”所有这样的后门路径
- Frontdoor Adjustment Set: 前门调整集
    - 如果一个变量集合 Z 相对于一个有序变量对 (X,Y) 满足前门准则： (1) Z 拦截了所有从 X 到 Y 的有向路径； (2) 从 X 到 Z 没有未被阻塞的后门路径； (3) 所有从 Z 到 Y 的后门路径都被 X 阻塞。 那么我们称 Z 为前门调整集，这个集合允许我们准确估计 X 对 Y 的因果效应。

### C-component